install github copilot


# Gather points by IN geologic formation


 
1) In this notebook, we will establish historical groundwater chemistry by geologic formation.
2) We will follow this by correlating historical precipitation to groundwater chemistry.
3) And finally, predicting groundwater chemistry change over time through different carbon emissions scenarios.

In [1]:
"""import libraries"""
from pykml import parser
import pandas as pd
import geopandas as gpd
from os import path
from shapely.geometry import Point

In [2]:

#read in GW chem data from IDNR
gw_filepath = r"C:\Users\janel\OneDrive\Documents\IU grad\Classes F23\EAS 690 Advanced Data Science\G690_Test\research_project\IDNR_GWchem\Data\GWCHEM_AA.xls"

# read in the data file, 
IDNR_gw_full = pd.read_excel(gw_filepath, sheet_name = "basins", header = 6, skiprows = 6)

# show the top of the dataset
IDNR_gw_full

,ref.no,pub.no,samp.no,UTMN,UTME,COUNTY,TOWNSHIP,RANGE,SECTION,DEPTH,...,LEAD,ARS,SEL,BOR,COMMENTS.1,Well type,Ref No.,Unnamed: 59,Sample #,Basin
0,85,NaN,WF-024,4451775,623650,18,20,9,6,165,...,NaN,NaN,NaN,NaN,NaN,B,85,NaN,WF-024,WF
1,1302,NaN,WF-002,4450598,632343,18,20,10,7,76,...,NaN,NaN,NaN,NaN,NaN,U,1302,NaN,WF-002,WF
2,1551,NaN,WF-003,4447960,637860,18,20,10,22,37,...,NaN,NaN,NaN,NaN,NaN,U,1551,NaN,WF-003,WF
3,2577,NaN,WF-014,4456595,627750,18,21,9,22,96,...,NaN,NaN,NaN,NaN,NaN,B,2577,NaN,WF-014,WF
4,2752,NaN,WF-011A,4459750,641860,18,21,10,12,121,...,NaN,NaN,NaN,NaN,NaN,B,2752,NaN,WF-011A,WF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,265132,132U,KK-079,4596450,534950,46,36,1,31,175,...,NaN,NaN,NaN,NaN,NaN,U,265132,NaN,KK-079,KK
1025,265226,5,SJ-021,4619950,562275,71,38,2,24,71,...,NaN,NaN,NaN,NaN,NaN,U,265226,NaN,SJ-021,SJ
1026,266940,23,SJ-020,4615625,565825,71,37,3,5,200,...,NaN,NaN,NaN,NaN,NaN,UNKNOWN,266940,NaN,SJ-020,SJ
1027,310076,110U,KK-168,4602125,522925,46,36,3,14,150,...,NaN,NaN,NaN,NaN,NaN,U,310076,NaN,KK-168,KK


Now pair UTM coordinates with their formation as determined in the KML file

pykml to shapely, then do geometry (point vs polygon)
try getting individual shapefiles from google earth . use shape.ly "point and polygon" check stackoverflow

Read in shapefile with geologic formation polygons

In [4]:
#download a shapefile

shapefile = path.join("/Users/janel/OneDrive/Documents/IU grad/Classes F23/EAS 690 Advanced Data Science/G690_Test/research_project/data/geo_units/IN_geol_poly.shp")

gpd.read_file(shapefile)

,STATE,ORIG_LABEL,SGMC_LABEL,UNIT_LINK,REF_ID,GENERALIZE,SRC_URL,URL,geometry
0,IN,D,D;0,IND;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-87.44982 41.13308, -87.45839 41.132..."
1,IN,D,D;0,IND;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-86.97157 40.77332, -86.97353 40.773..."
2,IN,D,D;0,IND;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-87.37957 40.75348, -87.38177 40.751..."
3,IN,D,D;0,IND;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-85.50110 39.84990, -85.50347 39.849..."
4,IN,D,D;0,IND;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-85.71591 38.79307, -85.71443 38.792..."
...,...,...,...,...,...,...,...,...,...
600,IN,S3,S3;0,INS3;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-87.25816 41.61673, -87.26003 41.614..."
601,IN,S3,S3;0,INS3;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-85.45559 41.14164, -85.45806 41.138..."
602,IN,S3,S3;0,INS3;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-87.52223 41.71193, -87.52016 41.708..."
603,IN,S3,S3;0,INS3;0,IN004,"Sedimentary, carbonate",http://maps.indiana.edu/layerGallery.html?cate...,https://mrdata.usgs.gov/geology/state/sgmc-uni...,"POLYGON ((-85.73187 39.56519, -85.74046 39.565..."


In [5]:
#create a geodataframe from the IN geological formations shapefile
geo_formations = gpd.GeoDataFrame.from_file(shapefile)

In [6]:
#Create a geodataframe from coordinates in IDNR excel file
IDNR_gw_full['coords'] = list(zip(IDNR_gw_full['UTMN'], IDNR_gw_full['UTME']))
IDNR_gw_full['coords'] = IDNR_gw_full['coords'].apply(Point)

IDNR_gw_pts = gpd.GeoDataFrame(IDNR_gw_full, geometry = 'coords', crs = geo_formations.crs)

In [7]:
pointsInPolygons = gpd.tools.sjoin(IDNR_gw_pts, geo_formations, predicate='within', how='left')

In [8]:
pointsInPolygons

,ref.no,pub.no,samp.no,UTMN,UTME,COUNTY,TOWNSHIP,RANGE,SECTION,DEPTH,...,coords,index_right,STATE,ORIG_LABEL,SGMC_LABEL,UNIT_LINK,REF_ID,GENERALIZE,SRC_URL,URL
0,85,NaN,WF-024,4451775,623650,18,20,9,6,165,...,POINT (4451775.00000 623650.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1302,NaN,WF-002,4450598,632343,18,20,10,7,76,...,POINT (4450598.00000 632343.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1551,NaN,WF-003,4447960,637860,18,20,10,22,37,...,POINT (4447960.00000 637860.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2577,NaN,WF-014,4456595,627750,18,21,9,22,96,...,POINT (4456595.00000 627750.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2752,NaN,WF-011A,4459750,641860,18,21,10,12,121,...,POINT (4459750.00000 641860.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,265132,132U,KK-079,4596450,534950,46,36,1,31,175,...,POINT (4596450.00000 534950.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1025,265226,5,SJ-021,4619950,562275,71,38,2,24,71,...,POINT (4619950.00000 562275.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1026,266940,23,SJ-020,4615625,565825,71,37,3,5,200,...,POINT (4615625.00000 565825.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1027,310076,110U,KK-168,4602125,522925,46,36,3,14,150,...,POINT (4602125.00000 522925.00000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""read in the data"""
from os import path

kml_file = path.join("/Users/janel/OneDrive/Documents/IU grad/Classes F23/EAS 690 Advanced Data Science/G690_Test/research_project/data/ingeol.kml")

with open(kml_file) as f:

    doc = parser.parse(f)

